In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(200, 16)

In [3]:
len(df[df.tiling_compatible == True]) 

29

In [4]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format     87
1         group_structure     40
2                    None     29
3  tile_generation_failed     24
4                 timeout     12
5          cant_open_file      8


In [11]:
df[df.incompatible_reason == 'tile_generation_failed']

,collection_concept_id,concept_id,data_center,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables,num_granules,processing_level,groups
34,C2368309102-GES_DISC,G2382933016-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/Precipitat...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('nscan', 'npi...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, scan_datetime, year, mon...",74457.0,2,None
37,C1239900189-GES_DISC,G1266816514-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/DeepBlueSe...,xarray,HDF5,h5,False,tile_generation_failed,No group found in store <fsspec.mapping.FSMap ...,https://staging.openveda.cloud/api/titiler-cmr...,aerosol_optical_thickness_550_land,"[aerosol_optical_thickness_550_land, aerosol_o...",68484.0,2,None
43,C2600303218-LAADS,G2798719671-LAADS,LAADS,s3://prod-lads/AERDB_L2_VIIRS_SNPP/AERDB_L2_VI...,xarray,NetCDF-4,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('Idx_Atrack',...",https://staging.openveda.cloud/api/titiler-cmr...,Aerosol_Optical_Thickness_550_Expected_Uncerta...,[Aerosol_Optical_Thickness_550_Expected_Uncert...,700979.0,2,None
52,C2600305692-LAADS,G3297987468-LAADS,LAADS,s3://prod-lads/AERDB_L2_VIIRS_NOAA20/AERDB_L2_...,xarray,NetCDF-4,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('Idx_Atrack',...",https://staging.openveda.cloud/api/titiler-cmr...,Aerosol_Optical_Thickness_550_Expected_Uncerta...,[Aerosol_Optical_Thickness_550_Expected_Uncert...,398825.0,2,None
68,C2368310484-GES_DISC,G2391365116-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/Precipitat...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('nscan', 'npi...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, scan_datetime, year, mon...",64518.0,2,None
70,C2368311020-GES_DISC,G2387401818-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/Precipitat...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('nscan', 'npi...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, scan_datetime, year, mon...",54543.0,2,None
80,C2368310204-GES_DISC,G2384678317-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/Precipitat...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('nscan', 'npi...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, scan_datetime, year, mon...",76298.0,2,None
85,C2408750690-LPCLOUD,G2925428658-LPCLOUD,LPDAAC,s3://lp-prod-protected/EMITL2ARFL.001/EMIT_L2A...,xarray,netCDF-4,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('downtrack', ...",https://staging.openveda.cloud/api/titiler-cmr...,reflectance,[reflectance],189411.0,2A,None
99,C2368308779-GES_DISC,G2382287558-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/Precipitat...,xarray,netCDF,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('nscan', 'npi...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, scan_datetime, year, mon...",37805.0,2,None
103,C2408009906-LPCLOUD,G3229502778-LPCLOUD,LPDAAC,s3://lp-prod-protected/EMITL1BRAD.001/EMIT_L1B...,xarray,netCDF-4,nc,False,tile_generation_failed,"Couldn't find X/Y dimensions in ('downtrack', ...",https://staging.openveda.cloud/api/titiler-cmr...,radiance,"[radiance, flat_field_update]",189271.0,1B,None


In [10]:
list(df[df.incompatible_reason == 'cant_open_file'].data_url)[1]

's3://gesdisc-cumulus-prod-protected/Precipitation/PRECIP_AMSRE_AQUA.1/2005/233/AQUA_AMSRE_FCDR2021_L2_V1_20050821-S143524-E161416.017554.V01E.nc'

In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
